<a href="https://colab.research.google.com/github/chineidu/NLP-Tutorial/blob/main/notebook/06_Transformers/05_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Semantic Search](https://huggingface.co/learn/nlp-course/chapter5/6?fw=pt)

In [1]:
import IPython

IPython.version_info

(7, 34, 0, '')

In [2]:
!pip install rich
!pip install transformers[torch]
!pip install torch datasets evaluate
!pip install black[jupyter]


### Formatter For Colab

[Run only once, at startup]

```text
- Connect to your drive
```

```python
from google.colab import drive
drive.mount("/content/drive")
```

```sh
Install black for jupyter

!pip install black[jupyter]
```

```text
- Restart kernel

[Then]

- Place your .ipynb file somewhere on your drive
```

```python
# Anytime you want format your code run:
!black /content/drive/MyDrive/YOUR_PATH/YOUR_NOTEBOOK.ipynb
```

```text
- Don't save your notebook, hit F5 to refresh the page
- Now save!
```

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Built-in library
import re
import json
from typing import Any, Dict, List, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
from pprint import pprint
import pandas as pd
from rich import print

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
# %load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

### Load Data

In [5]:
import datasets
from datasets import load_dataset
from datasets.dataset_dict import DatasetDict, Dataset

In [6]:
fp: str = "lewtun/github-issues"

issues_dataset: Dataset = load_dataset(fp, split="train")
issues_dataset

Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

In [7]:
print(issues_dataset[10])

{
    'url': 'https://api.github.com/repos/huggingface/datasets/issues/2944',
    'repository_url': 'https://api.github.com/repos/huggingface/datasets',
    'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/2944/labels{/name}',
    'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/2944/comments',
    'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/2944/events',
    'html_url': 'https://github.com/huggingface/datasets/issues/2944',
    'id': 1000544370,
    'node_id': 'I_kwDODunzps47oxhy',
    'number': 2944,
    'title': 'Add  `remove_columns` to `IterableDataset ` ',
    'user': {
        'login': 'cccntu',
        'id': 31893406,
        'node_id': 'MDQ6VXNlcjMxODkzNDA2',
        'avatar_url': 'https://avatars.githubusercontent.com/u/31893406?v=4',
        'gravatar_id': '',
        'url': 'https://api.github.com/users/cccntu',
        'html_url': 'https://github.com/cccntu',
        'followers_url': 'https://api.github.com/users/cccntu/followers',
        'following_url': 'https://api.github.com/users/cccntu/following{/other_user}',
        'gists_url': 'https://api.github.com/users/cccntu/gists{/gist_id}',
        'starred_url': 'https://api.github.com/users/cccntu/starred{/owner}{/repo}',
        'subscriptions_url': 'https://api.github.com/users/cccntu/subscriptions',
        'organizations_url': 'https://api.github.com/users/cccntu/orgs',
        'repos_url': 'https://api.github.com/users/cccntu/repos',
        'events_url': 'https://api.github.com/users/cccntu/events{/privacy}',
        'received_events_url': 'https://api.github.com/users/cccntu/received_events',
        'type': 'User',
        'site_admin': False
    },
    'labels': [
        {
            'id': 1935892871,
            'node_id': 'MDU6TGFiZWwxOTM1ODkyODcx',
            'url': 'https://api.github.com/repos/huggingface/datasets/labels/enhancement',
            'name': 'enhancement',
            'color': 'a2eeef',
            'default': True,
            'description': 'New feature or request'
        }
    ],
    'state': 'open',
    'locked': False,
    'assignee': None,
    'assignees': [],
    'milestone': None,
    'comments': [],
    'created_at': 1632110460000,
    'updated_at': 1632110460000,
    'closed_at': None,
    'author_association': 'CONTRIBUTOR',
    'active_lock_reason': None,
    'pull_request': None,
    'body': '**Is your feature request related to a problem? Please describe.**\r\nA clear and concise description 
of what the problem is.\r\n\r\n```python\r\nfrom datasets import load_dataset\r\ndataset = load_dataset("c4", 
\'realnewslike\', streaming =True, split=\'train\')\r\ndataset = 
dataset.remove_columns(\'url\')\r\n```\r\n```\r\nAttributeError: \'IterableDataset\' object has no attribute 
\'remove_columns\'\r\n```\r\n\r\n**Describe the solution you\'d like**\r\n\r\nIt would be nice to have 
`.remove_columns()` to match the `Datasets` api. \r\n\r\n\r\n**Describe alternatives you\'ve 
considered**\r\n\r\nThis can be done with a single call to `.map()`, \r\n\r\nI can try to help add this. 🤗',
    'timeline_url': 'https://api.github.com/repos/huggingface/datasets/issues/2944/timeline',
    'performed_via_github_app': None,
    'is_pull_request': False
}

In [8]:
print(issues_dataset.column_names)

[
    'url',
    'repository_url',
    'labels_url',
    'comments_url',
    'events_url',
    'html_url',
    'id',
    'node_id',
    'number',
    'title',
    'user',
    'labels',
    'state',
    'locked',
    'assignee',
    'assignees',
    'milestone',
    'comments',
    'created_at',
    'updated_at',
    'closed_at',
    'author_association',
    'active_lock_reason',
    'pull_request',
    'body',
    'timeline_url',
    'performed_via_github_app',
    'is_pull_request'
]

In [9]:
# The issues_dataset contains issues and pull requests.
# Select ONLY the issues
issues_dataset_1: Dataset = issues_dataset.filter(
    (lambda x: x.get("is_pull_request") == False and len(x.get("comments")) > 0),
)

print(f"Size if data BEFORE: {issues_dataset.num_rows}\n")

print(f"Size if data AFTER: {issues_dataset_1.num_rows}\n")

Size if data BEFORE: 3019

Size if data AFTER: 808

In [10]:
all_columns: list[str] = issues_dataset_1.column_names
columns_to_keep: list[str] = ["title", "body", "html_url", "comments"]
# Columns present in ONLY all_columns
columns_to_remove: set = set(columns_to_keep).symmetric_difference(set(all_columns))
print(columns_to_remove)

{
    'node_id',
    'user',
    'milestone',
    'repository_url',
    'id',
    'events_url',
    'labels_url',
    'locked',
    'number',
    'pull_request',
    'assignees',
    'state',
    'performed_via_github_app',
    'updated_at',
    'labels',
    'assignee',
    'created_at',
    'active_lock_reason',
    'url',
    'comments_url',
    'closed_at',
    'timeline_url',
    'is_pull_request',
    'author_association'
}

In [11]:
issues_dataset_1 = issues_dataset_1.remove_columns(column_names=columns_to_remove)
issues_dataset_1

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 808
})

In [12]:
issues_dataset_1.set_format("pandas")
df: pd.DataFrame = issues_dataset_1[:]
# OR df = issues_dataset_1.to_pandas()

df.head()

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues/2945,Protect master branch,"[Cool, I think we can do both :), @lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).]","After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is merged into the master branch\r\n - ..."
1,https://github.com/huggingface/datasets/issues/2943,Backwards compatibility broken for cached datasets that use `.filter()`,"[Hi ! I guess the caching mechanism should have considered the new `filter` to be different from the old one, and don't use cached results from the old `filter`.\r\nTo avoid other users from having this issue we could make the caching differentiate the two, what do you think ?, If it's easy enough to implement, then yes please 😄 But this issue can be low-priority, since I've only encountered it in a couple of `transformers` CI tests., Well it can cause issue with anyone that updates `datasets` and re-run some code that uses filter, so I'm creating a PR, I just merged a fix, let me know if...","## Describe the bug\r\nAfter upgrading to datasets `1.12.0`, some cached `.filter()` steps from `1.11.0` started failing with \r\n`ValueError: Keys mismatch: between {'indices': Value(dtype='uint64', id=None)} and {'file': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None), 'speaker_id': Value(dtype='int64', id=None), 'chapter_id': Value(dtype='int64', id=None), 'id': Value(dtype='string', id=None)}`\r\n\r\nRelated feature: https://github.com/huggingface/datasets/pull/2836\r\n\r\n:question: This is probably a `wontfix` bug, since it can be solved by simply cleaning the..."
2,https://github.com/huggingface/datasets/issues/2941,OSCAR unshuffled_original_ko: NonMatchingSplitsSizesError,[I tried `unshuffled_original_da` and it is also not working],"## Describe the bug\r\n\r\nCannot download OSCAR `unshuffled_original_ko` due to `NonMatchingSplitsSizesError`.\r\n\r\n## Steps to reproduce the bug\r\n\r\n```python\r\n>>> dataset = datasets.load_dataset('oscar', 'unshuffled_original_ko')\r\nNonMatchingSplitsSizesError: [{'expected': SplitInfo(name='train', num_bytes=25292102197, num_examples=7345075, dataset_name='oscar'), 'recorded': SplitInfo(name='train', num_bytes=25284578514, num_examples=7344907, dataset_name='oscar')}]\r\n```\r\n\r\n## Expected results\r\n\r\nLoading is successful.\r\n\r\n## Actual results\r\n\r\nLoading throws ab..."
3,https://github.com/huggingface/datasets/issues/2937,load_dataset using default cache on Windows causes PermissionError: [WinError 5] Access is denied,"[Hi @daqieq, thanks for reporting.\r\n\r\nUnfortunately, I was not able to reproduce this bug:\r\n```ipython\r\nIn [1]: from datasets import load_dataset\r\n ...: ds = load_dataset('wiki_bio')\r\nDownloading: 7.58kB [00:00, 26.3kB/s]\r\nDownloading: 2.71kB [00:00, ?B/s]\r\nUsing custom data configuration default\r\nDownloading and preparing dataset wiki_bio/default (download: 318.53 MiB, generated: 736.94 MiB, post-processed: Unknown size, total: 1.03 GiB) to C:\Users\username\.cache\huggingface\datasets\wiki_bio\default\\r\n1.1.0\5293ce565954ba965dada626f1e79684e98172d950371d266

In [13]:
print(df["comments"].iloc[0])

['Cool, I think we can do both :)'
 '@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen 
to protect the master branch only from **merge commits** (see update comment above), so no need to 
disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to 
the remote master branch; and eventually reverted without messing up the repo history).']

In [14]:
# There are two comments in this particular comment index
print(len(df["comments"].iloc[0]))

2

In [15]:
# Use the explode method to create a row for each comment in a given index.
# i.e. index 0 with 2 comments creates 2 rows and index 1 with 6 comments creates 6 rows, etc.
comments_df: pd.DataFrame = df.explode("comments", ignore_index=True)
comments_df.head(3)

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues/2945,Protect master branch,"Cool, I think we can do both :)","After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is merged into the master branch\r\n - ..."
1,https://github.com/huggingface/datasets/issues/2945,Protect master branch,"@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).","After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is merged into the master branch\r\n - ..."
2,https://github.com/huggingface/datasets/issues/2943,Backwards compatibility broken for cached datasets that use `.filter()`,"Hi ! I guess the caching mechanism should have considered the new `filter` to be different from the old one, and don't use cached results from the old `filter`.\r\nTo avoid other users from having this issue we could make the caching differentiate the two, what do you think ?","## Describe the bug\r\nAfter upgrading to datasets `1.12.0`, some cached `.filter()` steps from `1.11.0` started failing with \r\n`ValueError: Keys mismatch: between {'indices': Value(dtype='uint64', id=None)} and {'file': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None), 'speaker_id': Value(dtype='int64', id=None), 'chapter_id': Value(dtype='int64', id=None), 'id': Value(dtype='string', id=None)}`\r\n\r\nRelated feature: https://github.com/huggingface/datasets/pull/2836\r\n\r\n:question: This is probably a `wontfix` bug, since it can be solved by simply cleaning the..."


In [16]:
# Convert back to Dataset
comments_dataset: Dataset = Dataset.from_pandas(comments_df)
comments_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 2964
})

In [17]:
# Add n new columns
comments_dataset = comments_dataset.map(
    lambda x: {"comment_length": len(x.get("comments").split())}
)

comments_dataset

Map:   0%|          | 0/2964 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length'],
    num_rows: 2964
})

In [18]:
# Short comments
comments_dataset.sort("comment_length").select_columns(["comments", "comment_length"])[
    :10
]

{'comments': ['https://github.com/huggingface/datasets/blob/6c766f9115d686182d76b1b937cb27e099c45d68/src/datasets/builder.py#L179-L186',
  'https://github.com/huggingface/datasets/blob/6c766f9115d686182d76b1b937cb27e099c45d68/src/datasets/builder.py#L179-L186',
  '@albertvillanova ',
  'Thanks!',
  '#self-assign',
  '#take',
  '#take',
  '#self-assign',
  'Resolved',
  'Ty!'],
 'comment_length': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
# Drop short comments.
# i.e comments like: 'Thanks!', '#self-assign', etc.
comments_dataset = comments_dataset.filter(lambda x: x.get("comment_length") > 15)
comments_dataset

Filter:   0%|          | 0/2964 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length'],
    num_rows: 2175
})

In [20]:
# Concatenate the issue title, description (body), and comments together in a new text column.
def concat_data(example: dict[str, Any]) -> dict[str, Any]:
    """This is used to concatenate the title, body and comments together."""
    title: str = example.get("title")
    comments: str = example.get("comments")
    body: str = example.get("body")
    result = {"text": f"{title} \n {body} \n {comments}"}

    return result


comments_dataset = comments_dataset.map(concat_data)
comments_dataset

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text'],
    num_rows: 2175
})

In [21]:
comments_dataset[0]

{'html_url': 'https://github.com/huggingface/datasets/issues/2945',
 'title': 'Protect master branch',
 'comments': '@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).',
 'body': 'After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pul

## Create Text Embeddings

- This [table](https://www.sbert.net/docs/pretrained_models.html#model-overview) shows the model overview for sentence-transformers (open source) semantic search.

<br>

[![image.png](https://i.postimg.cc/MZLLL8TS/image.png)](https://postimg.cc/c6QTK2w9)

In [22]:
from transformers import AutoModel, AutoTokenizer

repo_name: str = "sentence-transformers"
checkpoint: str = f"{repo_name}/multi-qa-mpnet-base-dot-v1"

# Instantiate tokenizer
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Instantiate model
model: AutoModel = AutoModel.from_pretrained(checkpoint)

In [23]:
import torch

# To speed up the processing, push to a GPU
device_str: str = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device=device_str)

device

device(type='cuda')

In [24]:
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [25]:
# To represent each GitHub issue entry as a single vector, we need to pool or
# average the token embeddings. One popular approach is CLS pooling, where the last
# hidden state for the special [CLS] token is collected.


def cls_pooling(model_output):
    """To represent each GitHub issue entry as a single vector, pool or average the token embeddings.
    Using CLS pooling, where the last hidden state for the special [CLS] token is collected."""
    return model_output.last_hidden_state[:, 0]


# Create a helper function that will tokenize a list of documents, place the tensors
# on the GPU (if available), feed them to the model, and finally apply CLS pooling to the outputs:


def get_embeddings(text_list: list):
    """This is used to tokenize a list of documents, place the tensors
    on the GPU (if available), feed them to the model, and apply CLS pooling to the outputs."""
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    # Push to encoded input to the GPU (if available)
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [26]:
print(comments_dataset["text"][0])

Protect master branch 
 After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all 
commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:
- 00cc036fea7c7745cfe722360036ed306796a3f2
- 13ae8c98602bbad8197de3b9b425f4c78f582af1
- ...

I propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the 
future:
-  For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is 
merged into the master branch
  - Currently, simple merge commits are already disabled
  - I propose to disable rebase merging as well
- ~~Protect the master branch from direct pushes (to avoid accidentally pushing of merge commits)~~
  - ~~This protection would reject direct pushes to master branch~~
  - ~~If so, for each release (when we need to commit directly to the master branch), we should previously disable 
the protection and re-enable it again after the release~~
-  Protect the master branch only from direct pushing of **merge commits**
  - GitHub offers the possibility to protect the master branch only from merge commits (which are the ones that 
introduce all the commits from the feature branch into the master branch).
  - No need to disable/re-enable this protection on each release 

This purpose of this Issue is to open a discussion about this problem and to agree in a solution. 
 @lhoestq now the 2 are implemented.

Please note that for the the second protection, finally I have chosen to protect the master branch only from 
**merge commits** (see update comment above), so no need to disable/re-enable the protection on each release 
(direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted 
without messing up the repo history).

In [27]:
# Test that the function works by feeding it the first text entry in our corpus
# and inspecting the output shape;
embedding = get_embeddings(comments_dataset["text"][0])
embedding.shape

torch.Size([1, 768])

In [28]:
embedding

tensor([[-1.5532e-01, -1.0023e-01, -7.0321e-02, -7.9817e-02, -1.0425e-01,
         -1.8799e-01,  1.0403e-02,  2.7286e-01, -9.8488e-03, -8.4140e-02,
          2.9261e-01, -7.7875e-02, -1.3246e-01,  2.1589e-01, -5.6229e-02,
          1.7055e-01,  2.0032e-01, -4.0798e-02, -9.2333e-02,  2.7923e-02,
          2.2608e-02, -6.8703e-02,  8.3200e-02,  4.3630e-02, -1.8378e-01,
          5.4012e-03, -1.4812e-02,  1.5650e-01, -4.3977e-01, -4.6957e-01,
          1.4525e-01,  2.3224e-01,  2.5106e-02,  5.7053e-01, -1.0436e-04,
         -3.4550e-04,  1.6949e-01, -4.0601e-02, -1.4010e-01, -2.1546e-01,
         -4.9011e-01, -4.4480e-01, -7.0063e-02, -8.3543e-03,  1.2881e-01,
          5.2868e-02, -1.3250e-01,  2.3846e-02,  3.8485e-01,  1.0555e-01,
          2.3861e-01, -1.0285e-01,  1.8124e-01,  1.3769e-02,  3.9201e-01,
          5.4745e-01, -6.6634e-02,  3.0320e-01,  5.4018e-02,  6.7586e-04,
          6.1106e-02,  1.7411e-01, -8.2372e-02, -2.8667e-01,  2.2702e-01,
         -1.7799e-01,  5.1817e-01, -4.

In [29]:
# Get embeddings for the entire dataset

# The embeddings have been converted to NumPy arrays — that’s because 🤗 Datasets
# requires this format inorder to index them with FAISS.
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x.get("text")).detach().cpu().numpy()[0]}
)

embeddings_dataset

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text', 'embeddings'],
    num_rows: 2175
})

### FAISS For Efficient Similarity Search

```text
FAISS: Facebook AI Similarity Search.

- Dataset of embeddings needs a search mechanism.
- 🤗 Datasets offers a FAISS index for this purpose.
- FAISS is a library providing efficient algorithms for searching and clustering embedding vectors.
- FAISS creates an index data structure to find similar embeddings.
- Creating a FAISS index in 🤗 Datasets is straightforward using Dataset.add_faiss_index() function.
- Specify the desired column to index in the dataset.
```

In [30]:
!pip install faiss-gpu

In [31]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text', 'embeddings'],
    num_rows: 2175
})

In [32]:
# - Queries can be performed on the FAISS index using Dataset.get_nearest_examples() function.
# - Nearest neighbor lookup enables finding similar examples based on embeddings.
# - Embedding a question is the first step to test the functionality.

question: str = "How can I load a dataset offline?"
question_embedding: np.ndarray = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [35]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

scores

array([22.40664 , 22.893995, 24.148972, 24.555553, 25.505049],
      dtype=float32)

In [36]:
samples_df: pd.DataFrame = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [38]:
print(f"Shape of data: {samples_df.shape}\n")
samples_df.head()

Shpae of data: (5, 8)

,html_url,title,comments,body,comment_length,text,embeddings,scores
4,https://github.com/huggingface/datasets/issues/824,Discussion using datasets in offline mode,Requiring online connection is a deal breaker in some cases unfortunately so it'd be great if offline mode is added similar to how `transformers` loads models offline fine.\r\n\r\n@mandubian's second bullet point suggests that there's a workaround allowing you to use your offline (custom?) dataset with `datasets`. Could you please elaborate on how that should look like?,"`datasets.load_dataset(""csv"", ...)` breaks if you have no connection (There is already this issue https://github.com/huggingface/datasets/issues/761 about it). It seems to be the same for metrics too.\r\n\r\nI create this ticket to discuss a bit and gather what you have in mind or other propositions.\r\n\r\nHere are some points to open discussion:\r\n- if you want to prepare your code/datasets on your machine (having internet connexion) but run it on another offline machine (not having internet connexion), it won't work as is, even if you have all files locally on this machine.\r\n- AFAIK,...",57,"Discussion using datasets in offline mode \n `datasets.load_dataset(""csv"", ...)` breaks if you have no connection (There is already this issue https://github.com/huggingface/datasets/issues/761 about it). It seems to be the same for metrics too.\r\n\r\nI create this ticket to discuss a bit and gather what you have in mind or other propositions.\r\n\r\nHere are some points to open discussion:\r\n- if you want to prepare your code/datasets on your machine (having internet connexion) but run it on another offline machine (not having internet connexion), it won't work as is, even if you have a...","[-0.47318071126937866, 0.24578367173671722, -0.012630763463675976, 0.14121408760547638, 0.28335559368133545, -0.1470210701227188, 0.6012278199195862, 0.012776073068380356, 0.2687809467315674, 0.13127808272838593, -0.02403687685728073, -0.023985162377357483, -0.024662325158715248, 0.40210670232772827, -0.09124023467302322, -0.10213788598775864, -0.1722334325313568, 0.038877222687006, -0.1540050506591797, 0.13259492814540863, -0.14081068336963654, -0.12811940908432007, -0.3727093040943146, -0.028236284852027893, -0.16281437873840332, -0.1865873783826828, 0.009142007678747177, 0.1420215219259...",25.505049
3,https://github.com/huggingface/datasets/issues/824,Discussion using datasets in offline mode,"The local dataset builders (csv, text , json and pandas) are now part of the `datasets` package since #1726 :)\r\nYou can now use them offline\r\n```python\r\ndatasets = load_dataset('text', data_files=data_files)\r\n```\r\n\r\nWe'll do a new release soon","`datasets.load_dataset(""csv"", ...)` breaks if you have no connection (There is already this issue https://github.com/huggingface/datasets/issues/761 about it). It seems to be the same for metrics too.\r\n\r\nI create this ticket to discuss a bit and gather what you have in mind or other propositions.\r\n\r\nHere are some points to open discussion:\r\n- if you want to prepare your code/datasets on your machine (having internet connexion) but run it on another offline machine (not having internet connexion), it won't work as is, even if you have all files locally on this machine.\r\n- AFAIK,...",38,"Discussion using datasets in offline mode \n `datasets.load_dataset(""csv"", ...)` breaks if you have no connection (There is already this issue https://github.com/huggingface/datasets/issues/761 about it). It seems to be the same for metrics too.\r\n\r\nI create this ticket to discuss a bit and gather what you have in mind or other propositions.\r\n\r\nHere are some points to open discussion:\r\n- if you want to prepare your code/datasets on your machine (having internet connexion) but run it on another offline machine (not having internet connexion), it won't work as is, even if you have a...","[-0.44908538460731506, 0.20950642228126526, -0.05981910973787308, 0.12935049

In [40]:
for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.comments}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print(f"URL: {row.html_url}")
    print("=" * 50)
    print()

COMMENT: Requiring online connection is a deal breaker in some cases unfortunately so it'd be great if offline mode
is added similar to how `transformers` loads models offline fine.

@mandubian's second bullet point suggests that there's a workaround allowing you to use your offline (custom?) 
dataset with `datasets`. Could you please elaborate on how that should look like?

SCORE: 25.505048751831055

TITLE: Discussion using datasets in offline mode

URL: https://github.com/huggingface/datasets/issues/824

==================================================

...

COMMENT: here is my way to load a dataset offline, but it **requires** an online machine
1. (online machine)
```
import datasets
data = datasets.load_dataset(...)
data.save_to_disk(/YOUR/DATASET/DIR)
```
2. copy the dir from online to the offline machine
3. (offline machine)
```
import datasets
data = datasets.load_from_disk(/SAVED/DATA/DIR)
```

HTH.

SCORE: 22.406639099121094

TITLE: Discussion using datasets in offline mode

URL: https://github.com/huggingface/datasets/issues/824

==================================================

In [58]:
def get_query_matches(*, query: str, K: int) -> pd.DataFrame:
    """This is used to to display the top K matches to a query."""
    query_embedding: np.ndarray = get_embeddings([query]).cpu().detach().numpy()

    scores, results = embeddings_dataset.get_nearest_examples(
        "embeddings", query_embedding, k=K
    )
    results_df: pd.DataFrame = pd.DataFrame.from_dict(results)
    results_df["scores"] = scores
    results_df.sort_values("scores", ascending=False, inplace=True)

    for _, row in results_df.iterrows():
        print(f"COMMENT: {row.comments}")
        print(f"SCORE: {row.scores}")
        print(f"TITLE: {row.title}")
        print(f"URL: {row.html_url}")
        print("=" * 70)
        print()

    return results_df

In [53]:
df.iloc[255:260, [1, 2]]

,title,comments
255,visualization for cc100 is broken,"[This looks like an issue with the cc100 dataset itself but not sure\r\nDid you try loading cc100 on your machine ?, Hi\nloading works fine, but the viewer only is broken\nthanks\n\nOn Wed, Apr 7, 2021 at 12:17 PM Quentin Lhoest ***@***.***>\nwrote:\n\n> This looks like an issue with the cc100 dataset itself but not sure\n> Did you try loading cc100 on your machine ?\n>\n> —\n> You are receiving this because you authored the thread.\n> Reply to this email directly, view it on GitHub\n> <https://github.com/huggingface/datasets/issues/2162#issuecomment-814793809>,\n> or unsubscribe\n> <https..."
256,any possibility to download part of large datasets only?,"[Not yet but it’s on the short/mid-term roadmap (requested by many indeed)., oh, great, really awesome feature to have, thank you very much for the great, fabulous work, We'll work on dataset streaming soon. This should allow you to only load the examples you need ;), thanks a lot Quentin, this would be really really a great feature to have\n\nOn Wed, Apr 7, 2021 at 12:14 PM Quentin Lhoest ***@***.***>\nwrote:\n\n> We'll work on dataset streaming soon. This should allow you to only load\n> the examples you need ;)\n>\n> —\n> You are receiving this because you authored the thread.\n> Reply ..."
257,data_args.preprocessing_num_workers almost freezes,"[Hi.\r\nI cannot always reproduce this issue, and on later runs I did not see it so far. Sometimes also I set 8 processes but I see less being showed, is this normal, here only 5 are shown for 8 being set, thanks\r\n\r\n```\r\n#3: 11%|███████████████▊ | 172/1583 [00:46<06:21, 3.70ba/s]\r\n#4: 9%|█████████████▏ | 143/1583 [00:46<07..."
258,adding ccnet dataset,"[closing since I think this is cc100, just the name has been changed. thanks ]"
259,"viewer ""fake_news_english"" error",[Thanks for reporting !\r\nThe viewer doesn't have all the dependencies of the datasets. We may add openpyxl to be able to show this dataset properly]


In [54]:
df["title"].value_counts(normalize=True)

Protect master branch                                                                                                                                                                                                                                                           0.002475
Add WIT Dataset                                                                                                                                                                                                                                                                 0.002475
Backwards compatibility broken for cached datasets that use `.filter()`                                                                                                                                                                                                         0.002475
wmt datasets fail to load                                                                                                                                    

In [62]:
query: str = "How does processing in a batch work"
get_query_matches(query=query, K=3)

COMMENT: Hi Thomas,
what I do not get from documentation is that why when you set batched=True,
this is processed in batch, while data is not divided to batched
beforehand, basically this is a question on the documentation and I do not
get the batched=True, but sure, if you think this is more appropriate in
forum I will post it there.
thanks
Best
Rabeeh

On Tue, Nov 10, 2020 at 12:21 PM Thomas Wolf <notifications@github.com>
wrote:

> Hi I don’t think this is a request for a dataset like you labeled it.
>
> I also think this would be better suited for the forum at
> https://discuss.huggingface.co. we try to keep the issue for the repo for
> bug reports and new features/dataset requests and have usage questions
> discussed on the forum. Thanks.
>
> —
> You are receiving this because you authored the thread.
> Reply to this email directly, view it on GitHub
> <https://github.com/huggingface/datasets/issues/823#issuecomment-724639476>,
> or unsubscribe
> <https://github.com/notifications/unsubscribe-auth/ARPXHH4FIPFHVVUHANAE4F3SPEO2JANCNFSM4TQQVEXQ>
> .
>

SCORE: 30.65512466430664

TITLE: how processing in batch works in datasets

URL: https://github.com/huggingface/datasets/issues/823

======================================================================

...

SCORE: 30.65512466430664

TITLE: how processing in batch works in datasets

URL: https://github.com/huggingface/datasets/issues/823

======================================================================

,html_url,title,comments,body,comment_length,text,embeddings,scores
0,https://github.com/huggingface/datasets/issues/823,how processing in batch works in datasets,"Hi Thomas,\nwhat I do not get from documentation is that why when you set batched=True,\nthis is processed in batch, while data is not divided to batched\nbeforehand, basically this is a question on the documentation and I do not\nget the batched=True, but sure, if you think this is more appropriate in\nforum I will post it there.\nthanks\nBest\nRabeeh\n\nOn Tue, Nov 10, 2020 at 12:21 PM Thomas Wolf <notifications@github.com>\nwrote:\n\n> Hi I don’t think this is a request for a dataset like you labeled it.\n>\n> I also think this would be better suited for the forum at\n> https://discuss....","Hi,\r\nI need to process my datasets before it is passed to dataloader in batch, \r\nhere is my codes \r\n\r\n```\r\nclass AbstractTask(ABC):\r\n task_name: str = NotImplemented\r\n preprocessor: Callable = NotImplemented\r\n split_to_data_split: Mapping[str, str] = NotImplemented\r\n tokenizer: Callable = NotImplemented\r\n max_source_length: str = NotImplemented\r\n max_target_length: str = NotImplemented\r\n # TODO: should not be a task item, but cannot see other ways.\r\n tpu_num_cores: int = None\r\n\r\n # The arguments set are for all tasks and needs to be ...",167,"how processing in batch works in datasets \n Hi,\r\nI need to process my datasets before it is passed to dataloader in batch, \r\nhere is my codes \r\n\r\n```\r\nclass AbstractTask(ABC):\r\n task_name: str = NotImplemented\r\n preprocessor: Callable = NotImplemented\r\n split_to_data_split: Mapping[str, str] = NotImplemented\r\n tokenizer: Callable = NotImplemented\r\n max_source_length: str = NotImplemented\r\n max_target_length: str = NotImplemented\r\n # TODO: should not be a task item, but cannot see other ways.\r\n tpu_num_cores: int = None\r\n\r\n # The ar...","[-0.5023795366287231, -0.17439508438110352, -0.22950220108032227, 0.130157470703125, 0.1780354231595993, 0.03179476410150528, 0.29556161165237427, 0.1713045984506607, -0.1484704315662384, 0.1780766397714615, 0.009121731854975224, 0.12478089332580566, 0.06411994248628616, 0.24562396109104156, -0.056165214627981186, -0.03589145094156265, -0.07459743320941925, 0.048377472907304764, 0.1096092090010643, 0.007234200835227966, -0.36760956048965454, 0.06899745017290115, -0.4043073058128357, 0.05223430320620537, -0.22304658591747284, -0.26935648918151855, -0.061245325952768326, -0.03925704210996628...",30.655125
1,https://github.com/huggingface/datasets/issues/823,how processing in batch works in datasets,Yes the forum is perfect for that. You can post in the `datasets` section.\r\nThanks a lot!,"Hi,\r\nI need to process my datasets before it is passed to dataloader in batch, \r\nhere is my codes \r\n\r\n```\r\nclass AbstractTask(ABC):\r\n task_name: str = NotImplemented\r\n preprocessor: Callable = NotImplemented\r\n split_to_data_split: Mapping[str, str] = NotImplemented\r\n tokenizer: Callable = NotImplemented\r\n max_source_length: str = NotImplemented\r\n max_target_length: str = NotImplemented\r\n # TODO: should not be a task item, but cannot see other ways.\r\n tpu_num_cores: int = None\r\n\r\n # The arguments set are for all tasks and needs to be ...",17,"how processing in batch works in datasets \n Hi,\r\nI need to process my datasets before it is passed to dataloader in batch, \r\nhere is my codes \r\n\r\n```\r\nclass AbstractTask(ABC):\r\n task_name: str = NotImplemented\r\n preprocessor: Callable = NotImplemented\r\n split_to_data_split: Mapping[str, str] = NotImplemented\r\n tokenizer: Callable = NotImplemented\r\n max_source_length: str = NotImplemented\r\n max_target_length: str = NotImplemented\r\n # TODO: should not be a task item, but cannot see other ways.\r\n tpu_num_cores: int = None\r\n\r\n # The ar...","[-0.5023795366287231, -0.17439508438110352, -0.22950220108032227, 0.130157470703125, 0.1780354231595993, 0.03179476410150528, 0.29

In [64]:
query: str = "How can improve the documentation of datasets"
get_query_matches(query=query, K=3)

COMMENT: Yes we are working on the doc right now, should be in the next release quite soon.

SCORE: 30.260379791259766

TITLE: [Feature] Keep the list of labels of a dataset as metadata

URL: https://github.com/huggingface/datasets/issues/4

======================================================================

...

SCORE: 25.969202041625977

TITLE: [Feature] Keep the list of labels of a dataset as metadata

URL: https://github.com/huggingface/datasets/issues/4

======================================================================

,html_url,title,comments,body,comment_length,text,embeddings,scores
2,https://github.com/huggingface/datasets/issues/4,[Feature] Keep the list of labels of a dataset as metadata,"Yes we are working on the doc right now, should be in the next release quite soon.",It would be useful to keep the list of the labels of a dataset as metadata. Either directly in the `DatasetInfo` or in the Arrow metadata.,17,"[Feature] Keep the list of labels of a dataset as metadata \n It would be useful to keep the list of the labels of a dataset as metadata. Either directly in the `DatasetInfo` or in the Arrow metadata. \n Yes we are working on the doc right now, should be in the next release quite soon.","[0.04343941807746887, -0.036206990480422974, -0.15638597309589386, -0.052454523742198944, 0.12230357527732849, 0.2208477407693863, 0.2759694457054138, 0.17885375022888184, 0.00874372012913227, 0.22673830389976501, -0.157405287027359, 0.5626072287559509, -0.27078744769096375, 0.23665167391300201, -0.023262230679392815, 0.016595318913459778, -0.04746421426534653, 0.020963408052921295, 0.13064606487751007, -0.033688608556985855, -0.24152983725070953, -0.16987986862659454, 0.15129399299621582, -0.05320284143090248, -0.23283235728740692, -0.03940807282924652, 0.1785152107477188, -0.291641861200...",30.260380
1,https://github.com/huggingface/datasets/issues/4,[Feature] Keep the list of labels of a dataset as metadata,"Hi,\r\nI hope we could get a better documentation.\r\nIt took me more than 1 hour to found this way to get the label information.",It would be useful to keep the list of the labels of a dataset as metadata. Either directly in the `DatasetInfo` or in the Arrow metadata.,25,"[Feature] Keep the list of labels of a dataset as metadata \n It would be useful to keep the list of the labels of a dataset as metadata. Either directly in the `DatasetInfo` or in the Arrow metadata. \n Hi,\r\nI hope we could get a better documentation.\r\nIt took me more than 1 hour to found this way to get the label information.","[-0.0012372644850984216, -0.16487111151218414, -0.1444353461265564, 0.08951263874769211, 0.16622760891914368, 0.18495865166187286, 0.31819161772727966, 0.08626117557287216, -0.07907035201787949, 0.31925591826438904, -0.16471421718597412, 0.547156572341919, -0.20061591267585754, 0.24912923574447632, -0.10227081179618835, 0.005949097219854593, -0.15736478567123413, 0.07841548323631287, 0.158057302236557, -0.12615589797496796, -0.18062910437583923, -0.1693413257598877, 0.025151681154966354, 0.07299867272377014, -0.35703474283218384, -0.08059339225292206, 0.21164342761039734, -0.20556977391242...",28.216829
0,https://github.com/huggingface/datasets/issues/4,[Feature] Keep the list of labels of a dataset as metadata,This should be accessible now as a feature in dataset.info.features (and even have the mapping methods).,It would be useful to keep the list of the labels of a dataset as metadata. Either directly in the `DatasetInfo` or in the Arrow metadata.,16,[Feature] Keep the list of labels of a dataset as metadata \n It would be useful to keep the list of the labels of a dataset as metadata. Either directly in the `DatasetInfo` or in the Arrow metadata. \n This should be accessible now as a feature in dataset.info.features (and even have the mapping methods).,"[-0.049605146050453186, -0.09847134351730347, -0.17237889766693115, -0.02733141928911209, 0.16801317036151886, 0.16547058522701263, 0.32426437735557556, 0.23050767183303833, -0.04784256964921951, 0.277883917093277, -0.23027127981185913, 0.49288830161094666, -0.18443314731121063, 0.2717520594596863, -0.07220461964607239, 0.0264095701277256, -0.10095421969890594, 0.05305645242333412, 0.04141265153884888, -0.03478182852268219, -0.30807143449783325, -0.16353321075439453, 0.1349450945854187, 0.06438116729259491, -0.23750022053718567, -0.04607643187046051, 0.14126580953598022, -0.195450693368911...",25.969202
